# Part 1 - Getting the Data

In [ ]:

import os
import pandas as pd
import kagglehub
import requests
from PIL import Image
import torch
from transformers import CLIPProcessor, CLIPModel
from tqdm import tqdm
import numpy as np
import json
from io import BytesIO
import time


In [ ]:
# 📁 Step 3: Set Up Directories
os.makedirs("dataset/images", exist_ok=True)
os.makedirs("embeddings", exist_ok=True)
os.makedirs("vectordb", exist_ok=True)


In [ ]:
# 📄 Step 4: Download & Preprocess Dataset
print("Downloading Amazon Product Dataset 2020...")
path = kagglehub.dataset_download("promptcloud/amazon-product-dataset-2020")
path = os.path.join(os.path.join(path, "home"), "sdf")
print(f"Dataset path: {path}")

csv_file = os.listdir(path)[0]
data_file = os.path.join(path, csv_file)
df = pd.read_csv(data_file)

columns_to_use = ['Uniq Id', 'Product Name', 'Category', 'Selling Price', 'About Product', 'Product Specification', 'Image']
df = df[[col for col in columns_to_use if col in df.columns]]
df = df.dropna(subset=['Product Name']).fillna("")
df['description'] = df['Product Name'] + ' ' + df['About Product'] + ' ' + df['Product Specification']
df['description'] = df['description'].str.strip()

df.to_csv("dataset/preprocessed_data.csv", index=False)
print(f"Preprocessed data saved with {len(df)} products")


Dataset path: /kaggle/input/amazon-product-dataset-2020/home/sdf
Preprocessed data saved with 10002 products


In [ ]:
# 🤖 Step 5: Load CLIP Model and Tokenizer
device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = CLIPModel.from_pretrained("openai/clip-vit-base-patch32").to(device)
processor = CLIPProcessor.from_pretrained("openai/clip-vit-base-patch32")
tokenizer = processor.tokenizer


In [ ]:
# 🧰 Step 6: Define Helper Functions

def download_image(url, product_id, retries=3):
    for attempt in range(retries):
        try:
            response = requests.get(url, stream=True, timeout=10)
            if response.status_code == 200:
                img = Image.open(BytesIO(response.content)).convert('RGB')
                img_path = f"dataset/images/{product_id}.jpg"
                img.save(img_path)
                return img_path
        except Exception as e:
            print(f"Attempt {attempt + 1} failed for {url}: {str(e)}")
            time.sleep(2)
    return None

def generate_embeddings(row):
    product_id = row['Uniq Id']
    enhanced_text = (
        f"Product: {row['Product Name']}. "
        f"Category: {row['Category']}. "
        f"Price: {row['Selling Price']}. "
        f"Description: {row['About Product']}. "
        f"Specs: {row['Product Specification']}"
    )

    text_inputs = tokenizer(enhanced_text, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_embedding = model.get_text_features(**text_inputs).cpu().numpy().flatten()

    image_embeddings = []
    image_paths = []

    if pd.notna(row['Image']):
        urls = [url.strip() for url in row['Image'].split('|') if url.strip()]
        for i, url in enumerate(urls):
            if any(bad in url.lower() for bad in ['transparent-pixel', 'placeholder', 'no-image']):
                continue
            if url.startswith('https:/') and not url.startswith('https://'):
                url = url.replace('https:/', 'https://')
            elif url.startswith('http:/') and not url.startswith('http://'):
                url = url.replace('http:/', 'http://')

            img_path = f"dataset/images/{product_id}_{i}.jpg"
            if not os.path.exists(img_path):
                try:
                    response = requests.get(url, timeout=10)
                    if response.status_code == 200:
                        Image.open(BytesIO(response.content)).convert('RGB').save(img_path)
                except Exception as e:
                    print(f"Failed to download {url}: {str(e)}")
                    continue

            if os.path.exists(img_path):
                try:
                    image = Image.open(img_path)
                    image_inputs = processor(images=image, return_tensors="pt").to(device)
                    with torch.no_grad():
                        img_embedding = model.get_image_features(**image_inputs).cpu().numpy().flatten()
                    image_embeddings.append(img_embedding.tolist())
                    image_paths.append(img_path)
                except Exception as e:
                    print(f"Error processing {img_path}: {str(e)}")

    return {
        "product_id": product_id,
        "text_embedding": text_embedding.tolist(),
        "image_embeddings": image_embeddings,
        "image_paths": image_paths,
        "metadata": {
            "name": row['Product Name'],
            "category": row['Category'],
            "price": row['Selling Price']
        }
    }


In [ ]:
# 🧠 Step 7: Generate and Save All Embeddings
print(f"Generating embeddings for {len(df)} products...")
embeddings = []
failed_products = []

for _, row in tqdm(df.iterrows(), total=len(df)):
    try:
        embedding = generate_embeddings(row)
        embeddings.append(embedding)
    except Exception as e:
        print(f"Failed to process product {row['Uniq Id']}: {str(e)}")
        failed_products.append(row['Uniq Id'])

with open("embeddings/all_embeddings.json", "w") as f:
    json.dump(embeddings, f, indent=2)

print(f"\n✅ Done! Processed: {len(embeddings)}, Failed: {len(failed_products)}")
if failed_products:
    print("Failed product IDs:", failed_products)


Generating embeddings for 10002 products...


100%|██████████| 10002/10002 [29:35<00:00,  5.63it/s]



✅ Done! Processed: 10002, Failed: 0


# Part 2 -  Preparing and Creating VectorDB

In [ ]:
import os
import json
import pinecone
from tqdm import tqdm
import numpy as np
from sklearn.metrics import top_k_accuracy_score
from collections import defaultdict
import time

In [ ]:
# 🔑 Step 2: Set Configuration Variables
PINECONE_API_KEY = ''
INDEX_NAME = "multimodal"
EMBEDDING_DIM = 512
BATCH_SIZE = 100
EVAL_SAMPLE_SIZE = 100


In [ ]:
# 🔁 Step 3: Initialize Pinecone and Create Index If Not Exists
pc = pinecone.Pinecone(api_key=PINECONE_API_KEY)

if INDEX_NAME not in pc.list_indexes().names():
    print(f"Creating Pinecone index '{INDEX_NAME}'...")
    pc.create_index(
        name=INDEX_NAME,
        dimension=EMBEDDING_DIM,
        metric="cosine",
        spec=pinecone.ServerlessSpec(
            cloud="aws",
            region="us-east-1"
        )
    )
    print("✅ Index created!")

index = pc.Index(INDEX_NAME)


In [ ]:
# 📥 Step 4: Load Embedding Data
with open("embeddings/all_embeddings.json") as f:
    embedding_data = json.load(f)

print(f"Loaded {len(embedding_data)} embedding records.")


Loaded 10002 embedding records.


In [ ]:
index.delete(delete_all=True)

{}

In [ ]:
# 🛠️ Step 5: Prepare Vectors (Only If Index Is Empty)
stats = index.describe_index_stats()

vectors = []
if stats['total_vector_count'] == 0:
    print("Index is empty. Preparing vectors for upload...")

    for item in tqdm(embedding_data, desc="Preparing vectors"):
        # Text embedding
        vectors.append({
            'id': f"{item['product_id']}_text",
            'values': item['text_embedding'],
            'metadata': {**item['metadata'], "type": "text"}
        })

        # Image embeddings
        for i, emb in enumerate(item['image_embeddings']):
            vectors.append({
                'id': f"{item['product_id']}_img_{i}",
                'values': emb,
                'metadata': {**item['metadata'], "type": "image", "img_idx": i}
            })
else:
    print("Index already contains vectors. Skipping upload.")


Index is empty. Preparing vectors for upload...


Preparing vectors: 100%|██████████| 10002/10002 [00:00<00:00, 196043.87it/s]


In [ ]:
# Re-generate the vectors list
#vectors = prepare_vectors(embedding_data)

# Upload
for i in tqdm(range(0, len(vectors), BATCH_SIZE), desc="Upserting"):
    index.upsert(vectors=vectors[i:i + BATCH_SIZE])


Upserting: 100%|██████████| 440/440 [04:37<00:00,  1.58it/s]


In [ ]:
# 📊 Step 7: Evaluate Retrieval Performance
print("\n🔍 Running retrieval evaluation...")

# Map ID → embedding
id_to_emb = {
    f"{e['product_id']}_text": e['text_embedding']
    for e in embedding_data if 'text_embedding' in e
}
for e in embedding_data:
    if 'image_embeddings' in e:
        for i, emb in enumerate(e['image_embeddings']):
            id_to_emb[f"{e['product_id']}_img_{i}"] = emb

sample_size = min(EVAL_SAMPLE_SIZE, len(id_to_emb))
test_ids = np.random.choice(list(id_to_emb.keys()), size=sample_size, replace=False)

metrics = {'text': {'recall@1': 0, 'recall@5': 0, 'recall@10': 0},
           'image': {'recall@1': 0, 'recall@5': 0, 'recall@10': 0}}
total = 0

for vec_id in tqdm(test_ids, desc="Evaluating"):
    vec_type = 'text' if vec_id.endswith('_text') else 'image'
    query_vec = id_to_emb[vec_id]

    try:
        results = index.query(
            vector=query_vec,
            top_k=10,
            include_metadata=True,
            filter={"type": {"$eq": vec_type}}
        )

        retrieved_ids = [m['id'] for m in results['matches']]
        true_id = vec_id

        for k in [1, 5, 10]:
            if true_id in retrieved_ids[:k]:
                metrics[vec_type][f'recall@{k}'] += 1

        total += 1

    except Exception as e:
        print(f"Error evaluating {vec_id}: {str(e)}")



🔍 Running retrieval evaluation...


Evaluating: 100%|██████████| 100/100 [00:23<00:00,  4.23it/s]


In [ ]:
# 📈 Step 8: Display Evaluation Results
print("\n📌 Evaluation Results:")
for vec_type in ['text', 'image']:
    print(f"\n{vec_type.capitalize()} Retrieval:")
    for k in [1, 5, 10]:
        recall = metrics[vec_type][f'recall@{k}'] / total if total else 0
        print(f"Recall@{k}: {recall:.2%}")



📌 Evaluation Results:

Text Retrieval:
Recall@1: 30.00%
Recall@5: 30.00%
Recall@10: 30.00%

Image Retrieval:
Recall@1: 66.00%
Recall@5: 68.00%
Recall@10: 69.00%


In [ ]:
# 📦 Step 9: View Final Index Stats
print("\n📦 Final Pinecone Index Stats:")
print(index.describe_index_stats())



📦 Final Pinecone Index Stats:
{'dimension': 512,
 'index_fullness': 0.0,
 'metric': 'cosine',
 'namespaces': {'': {'vector_count': 43977}},
 'total_vector_count': 43977,
 'vector_type': 'dense'}


# Part 3 - Creating User Query Response

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch

LLM_MODEL = "microsoft/phi-2"

# Load model and tokenizer
tokenizer = AutoTokenizer.from_pretrained(LLM_MODEL)
phi_model = AutoModelForCausalLM.from_pretrained(
    LLM_MODEL,
    torch_dtype=torch.float32
)

# Create pipeline
phi_pipe = pipeline(
    "text-generation",
    model=phi_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    do_sample=True,
    temperature=0.7
)


tokenizer_config.json:   0%|          | 0.00/7.34k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/798k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.11M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/1.08k [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/564M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

Device set to use cuda:0


In [ ]:
def retrieve_and_respond_phi(query, top_k=5, vec_type='text'):
    # Embed query using CLIP
    text_inputs = processor(text=query, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        query_vec = clip_model.get_text_features(**text_inputs).cpu().numpy().flatten().tolist()

    # Pinecone retrieval
    results = index.query(
        vector=query_vec,
        top_k=top_k,
        include_metadata=True,
        filter={"type": {"$eq": vec_type}}
    )

    # Build product context
    retrieved_info = ""
    for match in results['matches']:
        md = match['metadata']
        retrieved_info += f"- Name: {md.get('name', 'N/A')}, Category: {md.get('category', 'N/A')}, Price: {md.get('price', 'N/A')}\n"

    prompt = f"""You are a helpful product assistant.

Product Info:
{retrieved_info}

Question:
{query}

Answer:"""
    print(prompt)
    # Generate response
    response = phi_pipe(prompt)[0]['generated_text']
    return response.split("Answer:")[-1].strip()


In [ ]:
response = retrieve_and_respond_phi("What are the features of the Samsung Galaxy S21?")
print("🧠 Phi-2 Response:\n", response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful product assistant.

Product Info:
- Name: -, Category: , Price: $13.00
- Name: What's In the Box, Category: Toys & Games | Games & Accessories, Price: $19.25
- Name: Tune Into Environmental Sounds, Category: Toys & Games | Kids' Electronics | Electronic Learning Toys, Price: $14.10
- Name: Spot It! (Color/Packaging May Vary), Category: Toys & Games | Games & Accessories | Card Games, Price: $9.99
- Name: Indie Boards & Cards Grifters Nexus Games, Category: Toys & Games | Games & Accessories, Price: $18.00


Question:
What are the features of the Samsung Galaxy S21?

Answer:
🧠 Phi-2 Response:
 The features of the Samsung Galaxy S21 are:
1. A camera with an ultra wide-angle lens
2. A 5nm Exynos 2100 processor
3. A 4,000mAh battery
4. A display that is at least 6.2 inches
5. It supports 5G
6. It has 128GB of internal storage
7. It has a choice of 128GB, 256GB, and 512GB of storage


In [ ]:
def answer_from_image_phi(uploaded_image):
    # Embed image with CLIP
    image_inputs = processor(images=uploaded_image, return_tensors="pt").to(device)
    with torch.no_grad():
        query_vec = clip_model.get_image_features(**image_inputs).cpu().numpy().flatten().tolist()

    # Search Pinecone for similar products
    results = index.query(
        vector=query_vec,
        top_k=5,
        include_metadata=True,
        filter={"type": {"$eq": "image"}}
    )

    # Build context from retrieved matches
    retrieved_info = ""
    for match in results['matches']:
        md = match['metadata']
        retrieved_info += f"- Name: {md.get('name', 'N/A')}, Category: {md.get('category', 'N/A')}, Price: {md.get('price', 'N/A')}\n"

    # Prompt for LLM
    prompt = f"""You are a helpful product assistant.

Based on the image provided and matching products found, answer the user's question:

Product Info:
{retrieved_info}

Question:
What is this product and what is it used for?

Answer:"""
    print(prompt)
    response = phi_pipe(prompt)[0]['generated_text']
    return response.split("Answer:")[-1].strip()


In [ ]:
image_path = "kitchenmixer.png"
image = Image.open(image_path).convert("RGB")

response = answer_from_image_phi(image)
print("🧠 Phi-2 Response:\n", response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful product assistant.

Based on the image provided and matching products found, answer the user's question:

Product Info:
- Name: Kraft Tool DC615 Glitter Gun, Category: Toys & Games | Arts & Crafts | Adhesives, Price: $84.58
- Name: Play Circle by Battat – Making Dinner for Eight Cookware Set – 21Piece Kids Play Pots & Pans Set – Pretend Play Kitchen Set for Kids Age 3 Years & Up, Category: Toys & Games | Dress Up & Pretend Play | Pretend Play, Price: $13.75
- Name: Wood Eats! Marvelous Mixer Playset with Sugar, Milk, Flour and Egg, Category: Toys & Games | Dress Up & Pretend Play | Pretend Play | Kitchen Toys | Kitchen Playsets, Price: $19.73
- Name: Hey! Play! Coffee Maker Toy Set- Pretend Kitchen Appliance for Play Espresso or Cappuccino Coffee Shop, Single Serve Brewer for Boys and Girls, Category: Toys & Games | Dress Up & Pretend Play | Pretend Play | Kitchen Toys | Kitchen Playsets, Price: $9.98
- Name: KidKraft Wooden Espresso Smoothie Set with Pretend Fruits &

In [ ]:
def retrieve_and_respond_multimodal(query_text, query_image, top_k=5):
    # --- Text embedding ---
    text_inputs = processor(text=query_text, return_tensors="pt", padding=True, truncation=True).to(device)
    with torch.no_grad():
        text_vec = clip_model.get_text_features(**text_inputs).cpu().numpy().flatten().tolist()

    # --- Image embedding ---
    image_inputs = processor(images=query_image, return_tensors="pt").to(device)
    with torch.no_grad():
        image_vec = clip_model.get_image_features(**image_inputs).cpu().numpy().flatten().tolist()

    # --- Query both ---
    text_results = index.query(vector=text_vec, top_k=top_k, include_metadata=True, filter={"type": {"$eq": "text"}})
    image_results = index.query(vector=image_vec, top_k=top_k, include_metadata=True, filter={"type": {"$eq": "image"}})

    # --- Combine metadata from both ---
    combined_metadata = []
    seen_ids = set()

    for match in text_results['matches'] + image_results['matches']:
        if match['id'] not in seen_ids:
            combined_metadata.append(match['metadata'])
            seen_ids.add(match['id'])

    # --- Format product context ---
    retrieved_info = ""
    for md in combined_metadata[:top_k]:  # limit to top_k combined
        retrieved_info += f"- Name: {md.get('name', 'N/A')}, Category: {md.get('category', 'N/A')}, Price: {md.get('price', 'N/A')}\n"

    # --- Prompt to Phi-2 ---
    prompt = f"""You are a helpful product assistant.

Product Info:
{retrieved_info}

Question:
{query_text}

Answer:"""
    print(prompt)
    response = phi_pipe(prompt)[0]['generated_text']
    return response.split("Answer:")[-1].strip()


In [ ]:
# Example text + image input
question = "Can you tell me what this product is and how it is used?"
image_path = "kitchenmixer.png"  # Replace with your actual image path
image = Image.open(image_path).convert("RGB")

# Get response
response = retrieve_and_respond_multimodal(question, image)
print("🧠 Multimodal Response:\n", response)


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful product assistant.

Product Info:
- Name: -, Category: , Price: $13.00
- Name: Tune Into Environmental Sounds, Category: Toys & Games | Kids' Electronics | Electronic Learning Toys, Price: $14.10
- Name: Forum Novelties 70935 Party Supplies, 30.5 x 1.3 x 25.4 Cm, As Shown, Category: Home & Kitchen | Home Décor | Home Décor Accents | Decorative Accessories | Decorative Signs & Plaques, Price: $10.59
- Name: What's In the Box, Category: Toys & Games | Games & Accessories, Price: $19.25
- Name: Edupress Fact or Fiction? Game (EP60268), Category: Toys & Games | Games & Accessories | Board Games, Price: $12.99


Question:
Can you tell me what this product is and how it is used?

Answer:
🧠 Multimodal Response:
 The 'Edupress Fact or Fiction?' Game (EP60268) is a game that helps with teaching children the difference between fact and fiction. It has a unique design and is priced at $12.99.


In [ ]:
from google.colab import files
file = files.upload()

Saving kitchenmixer.png to kitchenmixer.png


In [ ]:
from openai import OpenAI
import pandas as pd
from tqdm import tqdm

# ✅ Create client with API key
client = OpenAI(api_key="")

# Load test queries
queries_df = pd.read_csv("queries.csv")

# Initialize output columns
queries_df['retrieved_context'] = ""
queries_df['response'] = ""
queries_df['Relevance'] = 0
queries_df['Groundedness'] = 0
queries_df['Fluency'] = 0

# GPT-4 Evaluation function
def evaluate_with_gpt4(query, context, response_text):
    system_prompt = """You are an expert AI evaluator. Evaluate the chatbot's answer quality for an e-commerce assistant.

You must assign each of the following scores from 1 to 5:

- Relevance: 1 = completely off-topic, 3 = partially addresses the question, 5 = fully relevant.
- Groundedness: 1 = hallucinated/unsupported, 3 = somewhat supported, 5 = fully grounded in the context.
- Fluency: 1 = broken English, 3 = understandable but awkward, 5 = fluent and natural.

Be honest and use the full range of scores. Don't default to 1 or 5.

Respond ONLY in this JSON format:
{
  "Relevance": x,
  "Groundedness": y,
  "Fluency": z
}
"""


    user_prompt = f"""Question: {query}

Context:
{context}

Answer:
{response_text}"""

    try:
        response = client.chat.completions.create(
            model="gpt-4-turbo",
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user", "content": user_prompt}
            ],
            temperature=0
        )
        scores = eval(response.choices[0].message.content)
        return scores
    except Exception as e:
        print("Evaluation error:", e)
        return {"Relevance": 0, "Groundedness": 0, "Fluency": 0}

# Generate and evaluate responses
for i, row in tqdm(queries_df.iterrows(), total=len(queries_df)):
    q = row['questions']
    try:
        # Replace with your actual RAG function
        full_response = retrieve_and_respond_phi(q)
        context_snippet = full_response.split("Question:")[0].replace("Product Info:", "").strip()
        answer = full_response.split("Answer:")[-1].strip()
    except Exception as e:
        context_snippet = "ERROR"
        answer = "ERROR"
        print(f"Error generating response for query {i}: {e}")

    queries_df.at[i, 'retrieved_context'] = context_snippet
    queries_df.at[i, 'response'] = answer

    if answer != "ERROR":
        scores = evaluate_with_gpt4(q, context_snippet, answer)
        queries_df.at[i, 'Relevance'] = scores["Relevance"]
        queries_df.at[i, 'Groundedness'] = scores["Groundedness"]
        queries_df.at[i, 'Fluency'] = scores["Fluency"]

# Save final results
queries_df.to_csv("final_evaluated_queries.csv", index=False)
print("✅ Done! Results saved to final_evaluated_queries.csv")


  0%|          | 0/30 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful product assistant.

Product Info:
- Name: LEGO Technic Bugatti Chiron 42083 Race Car Building Kit and Engineering Toy, Adult Collectible Sports Car with Scale Model Engine (3599 Pieces), Category: Toys & Games | Building Toys, Price: $347.78
- Name: Bugatti Chiron 12V Ride On Car. Premium Convertible Bugatti Chiron Style Ride on Car Kids Music,Lights, Doors, RC, Category: Toys & Games | Tricycles, Scooters & Wagons | Ride-On Toys & Accessories, Price: $389.99
- Name: LEGO Speed Champions 1974 Porsche 911 Turbo 3.0 75895 Building Kit, New 2019 (179 Pieces), Category: Toys & Games | Building Toys | Building Sets, Price: $22.13
- Name: LEGO Technic Power Functions Motor Set 8293 (10 Pieces), Category: Toys & Games | Building Toys | Building Sets, Price: $28.82
- Name: LEGO Speed Champions 2018 Dodge Challenger SRT Demon and 1970 Dodge Charger R/T 75893 Building Kit, 2019 (478 Pieces), Category: Toys & Games | Building Toys | Building Sets, Price: $14.99 $ 14 . 99


Quest

  3%|▎         | 1/30 [00:03<01:38,  3.39s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful product assistant.

Product Info:
- Name: Delta Children Plastic Toddler Bed, Nickelodeon SpongeBob SquarePants, Category: Baby Products | Nursery | Furniture | Cribs & Nursery Beds | Toddler Beds, Price: $58.49
- Name: Delta Children Wood Toddler Bed, Nick Jr. PAW Patrol, Category: Home & Kitchen | Furniture | Kids' Furniture | Bed Frames, Headboards & Footboards, Price: $114.59
- Name: Nickelodeon Spongebob Divided Plate, Category: Baby Products | Feeding | Solid Feeding | Dishes, Price: $4.18
- Name: Child's Spongebob Squarepants Costume, Medium, Category: Clothing, Shoes & Jewelry | Costumes & Accessories | Kids & Baby | Girls, Price: $23.55
- Name: Delta Children Nolan Toy Box, Category: Baby Products | Nursery | Furniture | Storage & Organization | Toy Chests & Organizers, Price: $99.99


Question:
Is this Delta Children Toddler Bed themed with SpongeBob SquarePants?

Answer:


  7%|▋         | 2/30 [00:11<02:57,  6.34s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


...

You are a helpful product assistant.

Product Info:
- Name: Ped-Run Kids Pedaling Scooter - Exercise While Having Fun, Category: Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Scooters & Equipment | Scooters | Kick Scooters, Price: $103.46
- Name: Ped-Run Kids Pedaling Scooter - Exercise While Having Fun, Category: Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Scooters & Equipment | Scooters | Kick Scooters, Price: $103.44
- Name: 3 Wheeled Scooter for Kids - Stand & Cruise Child/Toddlers Toy Folding Kick Scooters w/Adjustable Height, Anti-Slip Deck, Flashing Wheel Lights, Category: Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Scooters & Equipment | Scooters, Price: $59.99
- Name: Ped-Run3 Kids Scooter for Boys and Girls with Advanced Dual Pedal Action, Category: Sports & Outdoors | Outdoor Recreation | Skates, Skateboards & Scooters | Scooters & Equipment | Scooters, Price: $124.66
- Name: Razor E Prim

 97%|█████████▋| 29/30 [02:38<00:06,  6.54s/it]Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


You are a helpful product assistant.

Product Info:
- Name: Noch 24205 Tree Summer 100-140Mm 10/ H0,Tt Scale Model Kit, Category: Toys & Games | Hobbies | Trains & Accessories | Accessories | Scenery, Price: $21.57
- Name: Noch 25113 Apple Tree 3-1/8" 3/ H0,Tt Scale Model Kit, Category: Toys & Games | Hobbies | Trains & Accessories | Accessories | Scenery, Price: $8.65
- Name: Noch 12850 in Agriculture W/Sound H0 Scale Model Kit, Category: Toys & Games | Hobbies | Trains & Accessories | Accessories | Scenery, Price: $28.63
- Name: Noch 65606 Forest Lodge H0 Scale, Category: Toys & Games | Hobbies | Trains & Accessories | Train Sets, Price: $39.59
- Name: Dead Fall Branches .5oz- 1/pkg, Category: Toys & Games | Hobbies | Trains & Accessories | Accessories | Scenery, Price: $7.99


Question:
How big is this “Noch Tree Summer” model kit for H0 scale?

Answer:


100%|██████████| 30/30 [02:47<00:00,  5.57s/it]

✅ Done! Results saved to final_evaluated_queries.csv


In [ ]:
files.download("final_evaluated_queries.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>